In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np
from nn_utils import load_mnist

In [2]:
#Dataset Directory
DATA_PATH = 'C:/Users/utilisateur/Desktop/1A CS/Pôle Projet (CycleGAN)/Datasets/mnist/'
LOGS_PATH = 'C:/Users/utilisateur/Desktop/1A CS/Pôle Projet (CycleGAN)/Logs/ann/'

# MNIST dataset parameters.
num_classes = 10 # total classes (0-9 digits).
num_features = 784 # data features (img shape: 28*28).

# Training parameters.
learning_rate = 0.001
training_steps = 3000
batch_size = 256
display_step = 100

# Network parameters.
n_hidden_1 = 32 # 1st layer number of neurons.
n_hidden_2 = 32 # 2nd layer number of neurons.

In [3]:
# Use tf.data API to shuffle and batch data.
(x_train, y_train), (x_test, y_test) = load_mnist(DATA_PATH, num_training=60000, num_test=10000, cnn=False, one_hot=False)
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [4]:
# Store layers weight & bias
# A random value generator to initialize weights.
random_normal = tf.initializers.RandomNormal()

weights = {
    'h1': tf.Variable(random_normal([num_features, n_hidden_1])),
    'h2': tf.Variable(random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.zeros([n_hidden_1])),
    'b2': tf.Variable(tf.zeros([n_hidden_2])),
    'out': tf.Variable(tf.zeros([num_classes]))
}

In [5]:
# Create model.
def neural_net(x):
    # Hidden fully connected layer with 128 neurons.
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Apply sigmoid to layer_1 output for non-linearity.
    layer_1 = tf.nn.sigmoid(layer_1)
    
    # Hidden fully connected layer with 256 neurons.
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Apply sigmoid to layer_2 output for non-linearity.
    layer_2 = tf.nn.sigmoid(layer_2)
    
    # Output fully connected layer with a neuron for each class.
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    # Apply softmax to normalize the logits to a probability distribution.
    return tf.nn.softmax(out_layer)

In [6]:
# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=num_classes)
    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # Compute cross-entropy.
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.Adam(learning_rate)

In [7]:
# Optimization process. 
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = neural_net(x)
        loss = cross_entropy(pred, y)
        
    # Variables to update, i.e. trainable variables.
    trainable_variables = list(weights.values()) + list(biases.values())

    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = LOGS_PATH + current_time + '/train'
test_log_dir = LOGS_PATH + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [8]:
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        test_pred = conv_net(x_test)
        test_loss = cross_entropy(test_pred, y_test)
        test_acc = accuracy(test_pred, y_test)

        with test_summary_writer.as_default():
            tf.summary.scalar('test_loss', test_loss, step=step)
            tf.summary.scalar('test_accuracy', test_acc, step=step)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 100, loss: 585.238037, accuracy: 0.148438
step: 200, loss: 585.683716, accuracy: 0.132812
step: 300, loss: 582.450378, accuracy: 0.117188
step: 400, loss: 567.200867, accuracy: 0.378906
step: 500, loss: 475.850098, accuracy: 0.406250
step: 600, loss: 394.423157, accuracy: 0.441406
step: 700, loss: 305.779388, accuracy: 0.667969
step: 800, loss: 226.784714, accuracy: 0.769531
step: 900, loss: 204.757278, accuracy: 0.750000
step: 1000, loss: 194.042358, accuracy: 0.800781
step: 1100, loss: 162.158905, accuracy: 0.839844
step: 1200, loss: 162.625443, accuracy: 0.800781
step: 1300, loss: 149.517029, accuracy: 0.843750
step: 1400, loss: 149.528580, accuracy: 0.875000
step: 1500, loss: 120.821320, accuracy: 0.875000
step: 1600, loss: 106.283676, accuracy: 0.882812
step: 1700, loss: 103.941666, accuracy: 0.894531
step: 1800, loss: 97.934265, accuracy: 0.890625
step: 1900, loss: 74.232811, accuracy: 0.914062
step: 2000, loss: 113.199615, accuracy: 0.878906
step: 2100, loss: 68.139023, ac

In [9]:
# Test model on validation set.
pred = neural_net(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.926300
